In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#imports
import numpy as np
import cv2
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Activation,Dense,Dropout,Flatten
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras import backend as Back
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
 
#constants
EPOCHS=25
INIT_LR=1e-3
BS=32
default_image_size=tuple((256,256))
image_size=0
directory_root='/content/gdrive/My Drive/PlantVillage'
width=256
height=256
depth=3
image_list,label_list=[],[]

In [ ]:
#convert image to array
def convertToArray(image_directory):
  try:
    img=cv2.imread(image_directory)
    if img is not None:
      img=cv2.resize(img,default_image_size)
      return img_to_array(img)
    else:
      return np.array([])
  except Exception as e:
    print(f"Exception:{e}")
    return None

In [ ]:
#loading images
try:
  #print(listdir('/content/gdrive/My Drive'))
 # print("[INFO] Loading Images")
  root=listdir(directory_root)
  
  #opens the plants in PlantVillage folder
  for plant in root:
    plant_disease_dir=listdir(f"{directory_root}/{plant}")
 
    #opens the diseases in the current plant folder
    for plant_disease in plant_disease_dir:
      #print(f"Processing {plant_disease}")
      plant_disease_image_dir=listdir(f"{directory_root}/{plant}/{plant_disease}/")
 
      #opens the images in the current plant/disease folder
      for disease_img in plant_disease_image_dir[:250]:
        img_dir=f"{directory_root}/{plant}/{plant_disease}/{disease_img}"
 
        #if it is an image convert to array and upate the image_list and label_list
        if img_dir.endswith(".jpg") or img_dir.endswith(".JPG"):
          image_list.append(convertToArray(img_dir))
          label_list.append(plant_disease)
#  print("[INFO] Completed")
 
except Exception as e:
  print(f"[ERROR]: {e}")

[INFO] Loading Images
Processing Tomato_Spider_mites_Two_spotted_spider_mite
Processing Tomato_Leaf_Mold
Processing Tomato__Tomato_mosaic_virus
Processing Tomato_Septoria_leaf_spot
Processing Tomato__Tomato_YellowLeaf__Curl_Virus
Processing Tomato_Late_blight
Processing Tomato__Target_Spot
Processing Tomato_healthy
Processing Tomato_Early_blight
Processing Tomato_Bacterial_spot
Processing Potato___healthy
Processing Potato___Early_blight
Processing Potato___Late_blight
Processing Pepper__bell___healthy
Processing Pepper__bell___Bacterial_spot
[INFO] Completed


In [ ]:
#print(len(image_list))
label_bin=LabelBinarizer()

3732


In [ ]:
labels=label_bin.fit_transform(label_list)
pickle.dump(label_bin,open('labeltransform.pkl','wb'))

In [ ]:
classesN=len(label_bin.classes_)
#print(classesN)

15


In [ ]:
np_image=np.array(image_list,dtype=np.float16)/225.0
#print(np_image[0][0])

In [ ]:
#Splitting train and test data
x_train,x_test,y_train,y_test=train_test_split(np_image,labels,test_size=0.2,random_state=42)
#print("Completed test train split")

In [ ]:
genImg=ImageDataGenerator(rotation_range=25,width_shift_range=0.1,height_shift_range=0.1,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,fill_mode="nearest")

In [ ]:
model=Sequential()
shape=(height,width,depth)
dim=-1
 
if Back.image_data_format()=="channels_first":
  shape=(depth,height,width)
  dim=1
 
model.add(Conv2D(32,(3,3),padding="same",input_shape=shape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=dim))
 
#downsampling  input
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))
 
model.add(Conv2D(64,(3,3),padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=dim))
model.add(Conv2D(32,(3,3),padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=dim))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
 
model.add(Conv2D(128,(3,3),padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=dim))
model.add(Conv2D(128,(3,3),padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=dim))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
#model.add(Dropout(0.5))
model.add(Dense(classesN))
model.add(Activation("softmax"))

In [ ]:
optimize=Adam(learning_rate=INIT_LR,decay=INIT_LR/EPOCHS)
 
#training thw model
model.compile(loss="binary_crossentropy",optimizer=optimize,metrics=['accuracy'])
#print("Training the model")

Training the model


In [ ]:
hist=model.fit_generator(genImg.flow(x_train,y_train,batch_size=BS)
,validation_data=(x_test,y_test),
steps_per_epoch=len(x_train)//BS,epochs=EPOCHS,verbose=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/25
93/93 [==============================] - 587s 6s/step - loss: 0.6221 - accuracy: 0.3442 - val_loss: 2.9779 - val_accuracy: 0.0428
Epoch 2/25
93/93 [==============================] - 590s 6s/step - loss: 0.1395 - accuracy: 0.6304 - val_loss: 1.1370 - val_accuracy: 0.0656
Epoch 3/25
93/93 [==============================] - 586s 6s/step - loss: 0.1630 - accuracy: 0.5204 - val_loss: 0.4529 - val_accuracy: 0.1017
Epoch 4/25
93/93 [==============================] - 590s 6s/step - loss: 0.1025 - accuracy: 0.7325 - val_loss: 0.5520 - val_accuracy: 0.0977
Epoch 5/25
93/93 [==============================] - 587s 6s/step - loss: 0.0947 - accuracy: 0.7448 - val_loss: 0.4459 - val_accuracy: 0.2142
Epoch 6/25
93/93 [==============================] - 590s 6s/step - loss: 0.0794 - accuracy: 0.8032 - val_loss: 0.2525 - val_accuracy: 0.4230
Epoch 7/25
93/93 [==============================] - 590s 6s/step - loss: 0.0704 - accuracy: 0.8253 - val_loss: 0.1743 - val_accuracy: 0.6037
Epoch 8/25
93

In [ ]:
'''acc=hist.history['accuracy']
val_acc=hist.history['val_accuracy']
los=hist.history['loss']
val_los=hist.history['val_loss']
 
epochs=range(1,len(acc)+1)
 
plt.plot(epochs,acc,'b',label="Training Accuracy")
plt.plot(epochs,val_acc,'r',label="Validation Accuracy")
plt.title('Training and Testing accuracies')
plt.legend()
 
plt.figure()
 
plt.plot(epochs,los,'b',label="Training Loss")
plt.plot(epochs,val_los,'r',label="Validation Loss")
plt.title('Training and Testing Losses')
plt.legend()
 
plt.show()'''

'acc=hist.history[\'accuracy\']\nval_acc=hist.history[\'val_accuracy\']\nlos=hist.history[\'loss\']\nval_los=hist.history[\'val_loss\']\n \nepochs=range(1,len(acc)+1)\n \nplt.plot(epochs,acc,\'b\',label="Training Accuracy")\nplt.plot(epochs,val_acc,\'r\',label="Validation Accuracy")\nplt.title(\'Training and Testing accuracies\')\nplt.legend()\n \nplt.figure()\n \nplt.plot(epochs,los,\'b\',label="Training Loss")\nplt.plot(epochs,val_los,\'r\',label="Validation Loss")\nplt.title(\'Training and Testing Losses\')\nplt.legend()\n \nplt.show()'

In [ ]:
score=model.evaluate(x_test,y_test)
print(f"Score: {score[1]*100.0}")

24/24 [==============================] - 30s 1s/step - loss: 0.0618 - accuracy: 0.8701
Score: 87.01472282409668


In [ ]:
model_json=model.to_json()
with open('cnn_model.json','w') as jsonf:
 jsonf.write(model_json)
 
model.save('model.h5')

from google.colab import files
files.download('cnn_model.json')
files.download('model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from keras import load_model
m=load_model('model.h5')

ImportError: ignored